In [2]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime

In [ ]:
class CrawlBrokerInfoTW:
    def __init__(self):
        self.target_name = "台股券商資訊"
        self.format = "non_time_series"

    @staticmethod
    def headquarter_info():
        r = requests.get('https://www.twse.com.tw/zh/brokerService/brokerServiceAudit')
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[0])
        df['department'] = '總公司'
        df = df.drop(columns='分公司')
        return df

    @staticmethod
    def branch_info(broker_hq_id):
        url = 'https://www.twse.com.tw/brokerService/brokerServiceAudit?showType=list&stkNo=' + broker_hq_id +\
              '&focus=6'
        r = requests.get(url)
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[3])
        return df

    def crawl_main(self):
        broker_hq = self.headquarter_info()
        branch_data = pd.concat([self.branch_info(i) for i in broker_hq['證券商代號'].values])
        branch_data['department'] = '分公司'
        df_all = pd.concat([broker_hq, branch_data])
        df_all = df_all.rename(columns={'證券商代號': 'stock_id', '證券商名稱': 'broker_name',
                                        '開業日': 'date_of_establishment', '地址': 'address',
                                        '電話': 'phone'
                                        })
        df_all=df_all[df_all['stock_id']!='查無資料']
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s.replace(' ',''))
        return df_all
    
a=CrawlBrokerInfoTW()
df=a.broker_all_info()
df
    
            
    

In [12]:
class GetNTLSxy:

    @classmethod
    def get_xy(cls, address):
        for i in ['新竹科學工業園區','新竹科學園區','大發工業區',
                  '南部科學工業園區','平鎮工業區','高雄加工出口區'
                  '南崗工業區']:
            address=address.replace(i,'')
            
        #解決郵遞區號問題
        filter_num = filter(str.isalpha, address[:6])
        address=''.join(list(filter_num))+address[6:]
            
        url = 'https://moisagis.moi.gov.tw/moiap/gis2010/content/user/matchservice/singleMatch.cfm'
        form = {
            'address': address,
            'matchRange': '0',
            'fuzzyNum': '0',
            'roadEQstreet': 'false',
            'subnumEQnum': 'false',
            'isLockTown': 'false',
            'isLockVillage': 'false',
            'ex_coor': 'EPSG:4326',
            'U02DataYear': '2015',
            'output_xml': '1'
        }
        try:
            r = requests.post(url, data=form)
            html_df = pd.read_html(StringIO(r.text))
        except ValueError:
            return None
        df = pd.DataFrame(html_df[0])
        df = df.where(pd.notnull(df), None)
        return df

    # 地址有些漏區的
    @classmethod
    def main_process(cls, address):
        df = cls.get_xy(address)
        if df is None:
            return None
        elif df['X'].values[0] is None:
            address = address[:3] + '信義區' + address[3:]
            df = cls.get_xy(address)
            return df
        else:
            return df

def update_xy_data(model_name,start=None,end=None):
    bulk_update_data = []
    obj_list = model_name.objects.all()[start:end]
    for obj_check in obj_list:
        location = obj_check.address
        print(location, obj_check.id)
        df = GetNTLSxy.main_process(location)
        if df is None:
            print('pass')
            continue
        obj_check.city = df['縣市'].values[0]
        obj_check.district = df['鄉鎮'].values[0]            
        obj_check.latitude = df['X'].values[0]
        obj_check.longitude = df['Y'].values[0]
        
        bulk_update_data.append(obj_check)
    update_fields_area = ['city','district','latitude', 'longitude']
    model_name.objects.bulk_update(bulk_update_data, update_fields_area, batch_size=1000)

# GetNTLSxy.main_process('台北市信義路4段236號4樓部分、5樓部分及7樓部分')

In [10]:
from crawlers.models import *
update_xy_data(BrokerInfoTW)

台北市大安區忠孝東路四段325號2樓(部分)、經紀部複委託科地址：台北市松山區長安東路二段225號5樓 2672
台北市延平南路八十一號 2673
台北市重慶南路1段58號4樓、5樓部分 2674
台北市塔城街30號4樓 2675
台北市南京東路2段111號3樓及5、6、7、8、12、13樓部分 2676
台北市衡陽路68號3樓 2677
台北市信義路4段236號4樓部分、5樓部分及7樓部分 2678
台北市復興南路1段2號5樓及5樓之1 2679
臺北市敦化南路二段95號27樓 2680
台北市敦化南路2段207號9樓部分、17樓部分 2681
台北市敦化南路2段207號22樓 2682
台北市敦化南路2段207號11樓 2683
台北市民生東路3段109號5樓 2684
台北市仁愛路4段296號13樓 2685
台北市松智路1號17樓部分 2686
台北市忠孝東路5段68號38樓部分 2687
台北市信義區松智路１號14樓(部分)、15樓(部分) 2688
台北市松仁路7號5樓部分 2689
台北市中正區重慶南路一段86號2樓 2690
臺北市中山區南京東路三段225號4樓之1、11樓及11樓之1、之2、之4、12樓及12樓之1、之2 2691
台北市大安區敦化南路2段97號32樓及地下1樓 2692
台北市承德路1段17號17樓、17樓之1、之2、之3 2693
台北市南京東路3段287號3樓部份 2694
台北市民生東路2段174、176號3樓部份、4樓、174號12樓 2695
桃園市中壢區中和路92.94.96.98號 2696
台北市中山區長安東路一段22號2樓部分及4樓 2697
台中市北屯區北興街43號1樓、2樓 2698
台北市南海路20號2樓部分、3樓部分 2699
苗栗縣苗栗市中正路426號4F 2700
台北市東興路8號1樓、2樓部分、3樓部分、5樓部分 2701
高雄市新興區七賢二路38號 2702
花蓮縣花蓮市公園路26號4,5,6樓 2703
台北市復興南路1段209號1、2、3樓 2704
南投縣草屯鎮草溪路1020號2、3樓 2705
台北市復興北路99號3樓、6樓及6樓之1 2706
台中市民族路45號1、2樓 2707
臺北市南港區經貿二路168號3樓 2708
高雄市前金區中華三路11號5樓及地下一樓 2709
彰化縣北斗鎮

雲林縣西螺鎮中山路127號3樓 3018
雲林縣斗南鎮中正路131號3樓 3019
雲林縣虎尾鎮公安路133號4樓 3020
台南縣麻豆鎮興中路127號3樓 3021
台北市金山南路2段33號2樓 3022
高雄市中正四路235號2樓之1 3023
台中市西區館前路55號1樓 3024
台南市新營區中營里中山路116號 3025
台北市中山北路6段126號3樓部分 3026
桃園縣桃園市成功路2段2號2樓 3027
新北市板橋區三民路1段216號2樓 3028
台北市南京東路3段65號3樓 3029
高雄市小港區漢民路381號2樓 3030
台北市民生西路286號3樓 3031
桃園市桃園區桃鶯路343號3、4樓 3032
新竹市中正路129號3樓 3033
台北市重慶南路1段2號2樓〈部分〉 3034
新北市永和區永和路2段121號3樓 3035
高雄市新光路36號1樓 3036
台北市忠孝東路4段563號3樓、565號4樓 3037
桃園市中壢區中山路142號8樓、146號8樓 3038
台北市民生東路3段128號3樓之2 3039
彰化縣彰化市和平路57號6樓 3040
彰化縣鹿港鎮彰鹿路8段59號 3041
台北市羅斯福路2段9號2樓(部分) 3042
高雄市大順二路41號2樓 3043
台北市信義區忠孝東路5段510號3樓、3樓之1、3樓之2、3樓之12及3樓之13 3044
台北市信義路3段182號3樓、4樓 3045
彰化縣員林鎮中山路1段753號3樓 3046
臺北市忠孝東路二段95號3樓 3047
台北市復興北路99號8樓之1 3048
台北市內湖區成功路4段166號5樓 3049
台中市西屯區臺灣大道4段600號3樓 3050
高雄縣岡山鎮中山北路28號3樓(部分) 3051
新北市新莊區思源路427號2樓 3052
台北市信義路4段96號2樓 3053
台南縣佳里鎮建南里九鄰光復路一四八號 3054
台南市府前路1段141號2、3樓 3055
台南市金華路二段一七二號地下一樓 3056
台北市信義路二段二一三號二樓 3057
高雄市光華一路148之83號1、2樓 3058
台北市南京東路五段一八八號四樓 3059
台南市東區中華東路3段242號1、2、3樓 3060
台東縣台東市更生路560號2樓及3樓 3061
台北市北投區北投路2段13號2

台中市西屯區西屯路2段256巷6號3樓之1 3372
台中市中區自由路2段8號3樓之1 3373
台北市八德路3段32號3樓 3374
台南市西門路1段223巷20號3樓 3375
臺南市永康區永大路2段179之3號2樓 3376
新竹市民生路276號2樓 3377
高雄市三民區九如一路73號2樓 3378
新北市鶯歌區國慶街22號 3379
新竹縣竹北市光明六路85號3樓 3380
高雄市路竹區竹南里國昌路66號 3381
南投縣南投市中山街202號4樓之2、之3、之5 3382
新北市新莊區中華路2段5號4樓 3383
新北市新莊區龍安路284號1樓及278巷3號2樓 3384
彰化縣彰化市華山路37號3樓 3385
臺北市士林區天母西路26之1號 3386
高雄市林園區鳳林路1段1號2樓 3387
高雄市楠梓區新昌街51號地下室、1、2樓 3388
苗栗縣苗栗市中正路462號4樓之1 3389
金門縣金城鎮民權路188號 3390
臺中市太平區中興東路62號3、4樓 3391
南投縣竹山鎮集山路3段919號2樓、橫街93號3樓 3392
台中市北屯區崇德路2段46之1號 3393
台北市忠孝東路1段85號3樓 3394
嘉義縣民雄鄉保生街212號 3395
台北市中正區新生南路1段148號4樓 3396
屏東縣潮州鎮延平路206號 3397
台南市歸仁區中山路2段29號3樓 3398
台南市佳里區和平街83號7樓 3399
台中市大里區國光路2段259號2樓 3400
屏東縣屏東市民生路287號5樓 3401
雲林縣西螺鎮延平路190、192號1、2樓 3402
台北市文林路342號2樓 3403
高雄市前金區中正四路143號4樓之1、4樓之2、5樓、8樓之1 3404
高雄市旗山區中山路102號2樓 3405
台北市松山區南京東路5段92號2樓 3406
台北市萬華區西園路1段200號地下1樓 3407
屏東縣屏東市濟南街10號 3408
台南市善化區大成路281號 3409
新竹市林森路196號2樓之3 3410
新北市土城區學府路1段25號1樓及地下1樓 3411
臺中市西區美村路一段272號2樓 3412
台南市新營區中山路117號2樓 3413
臺南市永康區小東路511號3樓 3414
宜蘭縣羅東鎮中正北路32號 3415
高雄市左營區明誠二路491

In [ ]:
from crawlers.models import *
bulk_update_data=[]
model_name=CompanyBasicInfoTW


# obj_check = model_name.objects.get(stock_id__contains='1020')


obj_list = model_name.objects.all()

for obj_check in obj_list[100:]:
    location=obj_check.address
    print(location,obj_check.id)
    df=GetNTLSxy.main_process(location)
    if df is None:
        print('pass')
        continue

    obj_check.latitude=df['X'].values[0]
    obj_check.longitude=df['Y'].values[0]
    bulk_update_data.append(obj_check)

update_fields_area = ['latitude','longitude']
model_name.objects.bulk_update(bulk_update_data, update_fields_area, batch_size=1000)




In [ ]:
a='南部科學工業園區台南市新市區大利一路6號'
a.replace('南部科學工業園區',"")

In [ ]:
address='南部科學工業園區台南市新市區大利一路6號'
for i in ['新竹科學工業園區','新竹科學園區','大發工業區','南部科學工業園區']:
    print(i)
    address=address.replace(i,'') 
address

In [ ]:
a='(114)台北市內湖區新湖一路151號7樓'
r=a.index(')')
if r<6:
    a=a[r+1:]
a

In [ ]:
address='(114)台北市內湖區新湖一路151號7樓'
filter_num = filter(str.isalpha, a[:6])
address=''.join(list(filter_num))+a[6:]
address

In [ ]:
a[:6]